# Text Generation using RNN

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
from tensorflow import keras

tf.config.optimizer.set_jit(True)

2024-03-17 14:01:35.824209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 14:01:35.824237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 14:01:35.825215: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load Shahnameh Dataset

In [2]:
df = pd.read_csv('./temp/shahname.csv')
df.head()

,Chapter,Part,Bait,Mesra,Text
0,1,1,1,1,به نام خداوند جان و خرد
1,1,1,1,2,کز این برتر اندیشه بر نگذرد
2,1,1,2,1,خداوند نام و خداوند جای
3,1,1,2,2,خداوند روزی ده رهنمای
4,1,1,3,1,خداوند کیوان و گَردان سپهر


In [3]:
text = '\n'.join(df['Text'])
print(text[:250])

به نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خداوند کیوان و گَردان سپهر
فروزندهٔ ماه و ناهید و مهر
ز نام و نشان و گمان برتر است
نگارندهٔ بر شده پیکر است
به بینندگان آفریننده را
نبینی مرنجان دو بینن


In [4]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

57 unique characters


In [5]:
for v in vocab:
    print(f'{v}', end=' ')


   ! ( ) :   « » ، ؛ ؟ ء آ أ ؤ ئ ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ل م ن ه و ي َ ُ ِ ّ ْ ٔ پ چ ژ ک گ ی ‌ 

In [6]:
# Defines a mapping from id to character, i.e. converts ids as integers into characters
ids_to_chars = keras.layers.StringLookup(vocabulary=vocab, invert=True, 
                                        mask_token=None)
# Defines a mapping from character to id, i.e. converts characters into ids as integers
ids_from_chars = keras.layers.StringLookup(vocabulary=vocab, invert=False, 
                                          mask_token=None)

In [7]:
# We have a UNK
for v in ids_from_chars.get_vocabulary():
    print(v, end=' ')

[UNK] 
   ! ( ) :   « » ، ؛ ؟ ء آ أ ؤ ئ ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ل م ن ه و ي َ ُ ِ ّ ْ ٔ پ چ ژ ک گ ی ‌ 

In [8]:
# Given a list of ids, it will return a string
def ids_to_text(ids):
    return tf.strings.reduce_join(ids_to_chars(ids), axis=-1)

print(ids_to_text([3, 4, 5, 6, 7, 8, 10, 12, 30, 33, 34]).numpy().decode('utf8'))

!(): «،؟شطظ


In [9]:
# Convert all of the text into numbers, i.e. ids
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
print(all_ids)

tf.Tensor([19 42  2 ... 27 56 41], shape=(2565041,), dtype=int64)


### Create a pipeline

In [10]:
SEQUENCE_LENGTH = 100
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
# EXAMPLE
for i in ids_dataset.take(10):
    print(i)

tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(42, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(41, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(40, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(24, shape=(), dtype=int64)
tf.Tensor(25, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)


In [12]:
sequences = ids_dataset.batch(batch_size=SEQUENCE_LENGTH + 1, drop_remainder=True, num_parallel_calls=AUTOTUNE)
# EXAMPLE
for ids in sequences.take(1):
    print(ids_to_text(ids).numpy().decode('utf-8'))

به نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خدا


#### Create ```(input, label)```

In [13]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list('Alireza Kamyab'))

(['A', 'l', 'i', 'r', 'e', 'z', 'a', ' ', 'K', 'a', 'm', 'y', 'a'],
 ['l', 'i', 'r', 'e', 'z', 'a', ' ', 'K', 'a', 'm', 'y', 'a', 'b'])

In [14]:
dataset = sequences.map(split_input_target, num_parallel_calls=AUTOTUNE)
# EXAMPLE
for input_example, target_example in dataset.take(1):
    print('Input:', ids_to_text(input_example).numpy().decode('utf-8'))
    print('Target:', ids_to_text(target_example).numpy().decode('utf-8'))

Input: به نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خد
Target: ه نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خدا


#### Creating training batches

In [15]:
# NOTE: Since we are creating a stateful model, we should NOT shuffle
dataset = dataset.batch(BATCH_SIZE, num_parallel_calls=AUTOTUNE, drop_remainder=True)
dataset = dataset.prefetch(AUTOTUNE)

In [16]:
# Length of vocabulary in StringLoopup Layer
VOCAB_SIZE = len(ids_from_chars.get_vocabulary())

# The embedding dimension
EMBEDDING_DIM = 10

# Number of RNN units
RNN_UNITS = 2048

In [17]:
class MyModel(keras.Model):
    def __init__(self, vocabulary_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding = keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embedding_dim)
        self.gru = keras.layers.GRU(units=rnn_units, return_sequences=True, return_state=True)
        self.dense = keras.layers.Dense(vocabulary_size)
        

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        if return_state:
            return x, states
        return x

In [18]:
model = MyModel(
    vocabulary_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNITS
)

#### Try the model

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 58) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  580       
                                                                 
 gru (GRU)                   multiple                  12656640  
                                                                 
 dense (Dense)               multiple                  118842    
                                                                 
Total params: 12776062 (48.74 MB)
Trainable params: 12776062 (48.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
sampled_indicies = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indicies = tf.squeeze(sampled_indicies, axis=-1).numpy()

In [22]:
sampled_indicies

array([53, 19, 46, 57, 14, 45,  3,  0, 18, 22, 10, 17,  1, 19, 55, 35, 35,
       51, 28, 10,  9, 37, 12, 38, 22, 17, 22, 56, 56, 39, 21, 39, 21, 56,
       38, 54, 20, 56, 27, 19, 39, 24, 29, 11, 36, 35, 31, 36, 54,  7, 33,
       15, 12, 34, 12, 25, 18, 38, 45, 22, 12, 50, 41, 26,  7, 18, 22, 41,
       24, 28,  2, 24, 46, 22, 39, 12,  3, 52, 11, 32, 19, 33, 24, 28, 50,
       38, 52, 24, 19, 51,  4, 52,  7, 31, 25,  1, 26, 15, 22, 25])

In [23]:
print('INPUT:',ids_to_text(input_example_batch[0]).numpy().decode('utf-8'))
print('NEXT CHAR:', ids_to_text(sampled_indicies).numpy().decode('utf-8'))

INPUT: به نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خد
NEXT CHAR: ژبُ‌آَ![UNK]اج،ئ
بگععپز،»ف؟قجئجییلثلثیقکتیربلخس؛غعصغک طأ؟ظ؟داقَج؟ٔنذ اجنخز خُجل؟!چ؛ضبطخزٔقچخبپ(چ صد
ذأجد


### Train the model

In [24]:
model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [25]:
checkpoints_dir = './temp/chpts/'
checkpoint_prefix = os.path.join(checkpoints_dir, 'chpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [26]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30


I0000 00:00:1710698505.229144   46236 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710698505.757496   46235 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


396/396 [==============================] - 50s 120ms/step - loss: 2.6338
Epoch 2/30
396/396 [==============================] - 47s 118ms/step - loss: 1.9455
Epoch 3/30
396/396 [==============================] - 47s 119ms/step - loss: 1.6312
Epoch 4/30
396/396 [==============================] - 47s 119ms/step - loss: 1.4627
Epoch 5/30
396/396 [==============================] - 47s 120ms/step - loss: 1.3492
Epoch 6/30
396/396 [==============================] - 47s 120ms/step - loss: 1.2553
Epoch 7/30
396/396 [==============================] - 47s 120ms/step - loss: 1.1630
Epoch 8/30
396/396 [==============================] - 47s 120ms/step - loss: 1.0639
Epoch 9/30
396/396 [==============================] - 48s 120ms/step - loss: 0.9702
Epoch 10/30
396/396 [==============================] - 48s 120ms/step - loss: 0.9075
Epoch 11/30
396/396 [==============================] - 48s 120ms/step - loss: 0.8690
Epoch 12/30
396/396 [==============================] - 48s 120ms/step - loss: 0.8325


### Generate Text

In [30]:
class OneStep(tf.keras.Model):
    def __init__(self, model, ids_to_chars, ids_from_chars, temperature=1.0):
        super(OneStep, self).__init__()
        self.temperature = temperature
        self.model = model
        self.ids_to_chars = ids_to_chars
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated
        ## This "[:, None]" thing is used to add another dim to skip_ids
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put -inf at each bad index
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model
        ## Predicted logits shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)

        # Only use the last prediction
        predicted_logits = predicted_logits[:, -1, :] # This means last sequence (i.e. last character)
        predicted_logits = predicted_logits/self.temperature

        # Apply the prediction mask
        predicted_logits += self.prediction_mask

        # Sample the output logits to generate token IDs
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.ids_to_chars(predicted_ids)

        # Return the characters and model states
        return predicted_chars, states

In [33]:
one_step_model = OneStep(model, ids_to_chars, ids_from_chars, temperature=0.75)

In [34]:
start = time.time()
states = None
next_char = tf.constant(['به نام خداوند'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n\n')
print('\nRun time:', end - start)

به نام خداوند جای نشای
چو مهتان یکی شاه خیره بمان
بدرید داننده شهر دار
کز آن ماه‌ای کار میساد و وار
ز باغار مردم که کردی تبار
همیشه بزرگان و گاه مهان
به هر جایگاهی رکان اندر اوی
شنیدی که بر خارست بود پاک
تو شادان و خونی مکن ز آختا
بران لشکر گشنه آن تیز کم
جهاندار خیزد میان به زس
ببینوش پس گفت کای پادشا
من این نامه در کار گردی تو گرد
همه دل نخواهد شکند از میان
چنین گفت کاین رنج داری به رزم
که هر بود روزی به من بر گذشت
مجای آن از اندیشه بر مرد دست
تو گردیم از ایشان سراسر بخرس
هر آنکس که جانم به تخت و کلاه
توبر گشتم اندر جهان کس نبود
که خورشید تابان مرا رفته بود
ازان کاخ آشای تا پاسخ نهزد
همی رفت پویان ز بی‌تند و دود
یکی داغ دیگر سر اندر فریب
همه بودنیها که و تن به تیر
به مشت و چباغ و به چرخ بلند
ز تمریکی و دهد زنهار درد
چنین گفت شیروی و پاغوز خال
نگویی مرا نیز فرخ مباد
همه مهربان موبد ایزدیست
کجا اندرین باره چرزی کسی
نخستین مه و مرگ نا باد سرد
ببادا به من بر به زندان بود
بگرد جهان تنگ و زیرد مبود
سر بدسگاران برو ناگزیر
نهانی نیاطت به روز نبرد
به خاک اندر آرم ز بهرام دید
شد از جادوی با سپ